In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from sklearn.model_selection import train_test_split

input_dir = '/kaggle/input/titanic/'

In [ ]:
def read_and_preprocess_csv(titanic_dir):
    '''
    Reads and preprocesses train and test csv's

    Args:
        titanic_dir (str): directory of the titanic dataset

    Returns:
        X_train (pd.DataFrame): input for the model training
        y_train (pd.Series): labels for training
        X_test (pd.DataFrame): input for the model testing
    '''
    train_csv = pd.read_csv(os.path.join(titanic_dir, 'train.csv'))
    test_csv = pd.read_csv(os.path.join(titanic_dir, 'test.csv'))
    
    ground_truth = 'Survived'
    features = pd.concat(
        (train_csv.drop(columns=['PassengerId', ground_truth]),
         test_csv.drop(columns=['PassengerId'])))
    numerical_features = features.dtypes[features.dtypes != 'object'].index
    
    # print(numerical_features)
    
    features[numerical_features] = features[numerical_features].fillna(0)

    # apply standardization to features (x - x.mean() / x.std())
    scaler = StandardScaler()
    features[numerical_features] = scaler.fit_transform(features[numerical_features])
    
    print(f'Shape before get_dummies: {features.shape}')
    features = pd.get_dummies(features, dummy_na=True)
    print(f'Shape after get_dummies: {features.shape}')
    # print(features.dtypes[features.dtypes == 'object'])

    features = features.astype('float32')
    
    y_train = train_csv[ground_truth]
    
    X_train = features.iloc[:len(y_train)]
    X_test = features.iloc[len(y_train):]
    
    return X_train, y_train, X_test
    
X_train_df, y_train_df, X_test_df = read_and_preprocess_csv(os.path.join(input_dir))

print(X_train_df.shape,y_train_df.shape, X_test_df.shape)
X_train_df

In [ ]:
X_train_df, X_valid_df, y_train_df, y_valid_df = train_test_split(X_train_df, y_train_df, test_size=0.2, random_state=1)

X_train = torch.tensor(X_train_df.values, dtype=torch.float32)
y_train = torch.tensor(y_train_df.values, dtype=torch.float32).unsqueeze(1)
X_valid = torch.tensor(X_valid_df.values, dtype=torch.float32)
y_valid = torch.tensor(y_valid_df.values, dtype=torch.float32).unsqueeze(1)
print(X_train.shape, y_train.shape)
train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(X_valid, y_valid)
batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)


In [ ]:
class CustomMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2437, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 64),
            nn.LeakyReLU(0.1),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

def get_gpus():
    gpu_count = torch.cuda.device_count()
    return [torch.device(f'cuda:{i}') for i in range(gpu_count)]

devices = get_gpus()
print(devices)

net = CustomMLP()



In [ ]:
def train(net, dataloader, num_epochs, lr, lr_period, lr_decay, momentum, devices):
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])

    loss = nn.BCELoss()
    optim = torch.optim.SGD(net.parameters(), lr=lr, momentum=momentum) 

    scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=lr_period, gamma=lr_decay)
    
    for epoch in range(num_epochs):
        net.train()
        for X, y in dataloader:
            X, y = X.to(devices[0]), y.to(torch.float32).to(devices[0])
            optim.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optim.step()
        scheduler.step()
        if (epoch+1) % 5 == 0:
            print(f'Epoch {epoch + 1} Loss: {l.item():.4f}')


train(net, train_loader, 100, lr=0.1, momentum=0.9, devices=devices, lr_period=5, lr_decay=0.99)
        

In [ ]:
def eval_acc(net, dataloader, devices):
    net.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(devices[0])
            y = y.to(devices[0])
            outputs = net(X)
            preds = (outputs > 0.5).float()
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total

acc = eval_acc(net, valid_loader, devices)
print(f'Validation accuracy: {acc:.4f}')

In [ ]:
X_test_tensor = torch.tensor(X_test_df.values, dtype=torch.float32).to(devices[0])
net.eval()
with torch.no_grad():
    preds = net(X_test_tensor)
    preds = (preds > 0.5).int().squeeze().cpu().numpy()

passenger_ids = pd.read_csv(os.path.join(input_dir, 'test.csv'))['PassengerId']

submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': preds
})
submission.to_csv('submission.csv', index=False)